## Train Results

In [5]:
import pandas as pd

# Load the CSV file (replace 'your_file.csv' with the actual path)
train_results_path = './results/240364_train_fusion_rates_results.csv'
train_id = train_results_path.split('/')[-1].split('_')[0]
print(f"Processing train id: {train_id}")

df = pd.read_csv(train_results_path)

# Clean the 'subject_id' column
df['subject_id'] = df['subject_id'].str.strip("[]").str.replace("'", "", regex=False)

# drop trial_id column
df = df.drop(columns=['trial_id'])

# Convert rate columns to numeric, coercing errors to NaN
rate_cols = ['smartwatch_rate', 'video_rate', 'gt_rate']
df[rate_cols] = df[rate_cols].apply(pd.to_numeric, errors='coerce')

# Drop rows with invalid subject_ids (e.g., '...' or empty)
print(df['subject_id'].unique())
# Compute subject-wise average for each rate
subject_avg = df.groupby('subject_id')[rate_cols].mean().reset_index()

# drop subject_id P10
subject_avg = subject_avg[subject_avg['subject_id'] != 'P10']

# drop index column
subject_avg = subject_avg.reset_index(drop=True)
# Print or save the result

print(subject_avg)

subject_avg.to_csv(f'./results/{train_id}_train_subject_wise_avg_rates.csv', index=False)  # Optional: save to CSV


Processing train id: 240364
['wa1' 'ng5' 'ng8' 'ng3' 'P14' 'ng4' 'ms2' 'P20' 'P5' 'P0' 'P12' 'ng6'
 'P21' 'P16' 'ng1' 'P11' 'P3' 'ng9' 'P4' 'P7' 'ms1' 'P22' 'P1' 'ng2' 'P2'
 'P15' 'P10']
   subject_id  smartwatch_rate  video_rate     gt_rate
0          P0       126.090909   57.545455  128.000000
1          P1       116.766667   57.633333  104.833333
2         P11       129.375000   83.475000  131.775000
3         P12       108.000000   36.000000  132.000000
4         P14       103.800000   51.600000   63.600000
5         P15       119.560606   51.641414  112.090909
6         P16       102.000000   78.000000  100.363636
7          P2       138.312500   80.500000  177.187500
8         P20       108.571429   58.857143  126.285714
9         P21       118.878409   72.367045  110.345455
10        P22       118.106061   75.280303  111.340909
11         P3       137.606061   68.878788  160.818182
12         P4       150.000000  106.666667  171.333333
13         P5       123.200000   65.450000 

## Test Result

In [6]:
results_dir = './results/fusion_test_999381'
window_wise_results_file_path = f'{results_dir}/debug_best_smartwatch_model_999381_test_fusion_debug_per_batch.csv'
subject_wise_results_file_path = f'{results_dir}/test_results_best_smartwatch_model_999381_test_fusion_rmse_per_batch.csv'

In [7]:
import pandas as pd
import numpy as np

# Load the CSV file (replace with your actual path if needed)
test_id = subject_wise_results_file_path.split('/')[-1].split('_')[0]
print(f"Processing test id: {test_id}")

test_result_folder = subject_wise_results_file_path.split('/')[-2]

df = pd.read_csv(subject_wise_results_file_path)

# Clean the 'subject_id' column
df['subject_id'] = df['subject_id'].str.strip("[]").str.replace("'", "", regex=False)

# Drop the trial_id column
df = df.drop(columns=['trial_id'])

# Ensure rate columns are numeric
rate_cols = [
    'rmse_cpm',
    'smartwatch_rate', 'video_rate', 'fused_rate', 'gt_rate',
    'smartwatch_depth', 'video_depth', 'fused_depth', 'gt_depth', 'rmse_depth'
]
df[rate_cols] = df[rate_cols].apply(pd.to_numeric, errors='coerce')

# --- NEW: compute simple 50/50 fusion rate per row ---
df['simple_fusion_rate'] = 0.5 * df['smartwatch_rate'] + 0.5 * df['video_rate']

# SIMPLE 50/50 FUSION DEPTH
df['simple_fusion_depth'] = 0.5 * df['smartwatch_depth'] + 0.5 * df['video_depth']

# Drop any rows with invalid or empty subject_ids
df = df[df['subject_id'].notna() & (df['subject_id'] != '')]

# Compute subject-wise averages, now including simple_fusion_rate
subject_avg = (
    df
    .groupby('subject_id', as_index=False)
    .agg({
        'rmse_cpm': 'mean',
        'fused_rate': 'mean',
        'simple_fusion_rate': 'mean',   # ← new
        'smartwatch_rate': 'mean',
        'video_rate': 'mean',
        'gt_rate': 'mean',
        'smartwatch_depth': 'mean',
        'video_depth': 'mean',
        'fused_depth': 'mean',
        'simple_fusion_depth': 'mean',  # ← new
        'gt_depth': 'mean',
        'rmse_depth': 'mean'
    })
    .rename(columns={
        'rmse_cpm': 'avg_fused_rmse_cpm',
        'fused_rate': 'avg_fused_rate',
        'simple_fusion_rate': 'avg_simple_fusion_rate',  # ← new
        'smartwatch_rate': 'avg_smartwatch_rate',
        'video_rate': 'avg_video_rate',
        'gt_rate': 'avg_gt_rate',
        'smartwatch_depth': 'avg_smartwatch_depth',
        'video_depth': 'avg_video_depth',
        'fused_depth': 'avg_fused_depth',
        'simple_fusion_depth': 'avg_simple_fusion_depth',  # ← new
        'gt_depth': 'avg_gt_depth',
        'rmse_depth': 'avg_rmse_depth'
    })
)

# Drop subject_id P10 if present
subject_avg = subject_avg[subject_avg['subject_id'] != 'P10']

# function to compute rmse between two series
def compute_rmse(a, b):
    return np.sqrt(np.mean((a - b) ** 2))

# Compute per-subject RMSEs, including simple_fusion
rmse_metrics = (
    df
    .groupby('subject_id')
    .apply(lambda g: pd.Series({
        'rmse_smartwatch_cpm':    compute_rmse(g['smartwatch_rate'], g['gt_rate']),
        'rmse_video_cpm':         compute_rmse(g['video_rate'],     g['gt_rate']),
        'rmse_fused_cpm':         compute_rmse(g['fused_rate'],     g['gt_rate']),
        'rmse_simple_fusion_cpm': compute_rmse(g['simple_fusion_rate'], g['gt_rate']),  # ← new
        'rmse_smartwatch_depth':  compute_rmse(g['smartwatch_depth'], g['gt_depth']),
        'rmse_video_depth':       compute_rmse(g['video_depth'],     g['gt_depth']),
        'rmse_fused_depth':       compute_rmse(g['fused_depth'],     g['gt_depth']),
        'rmse_simple_fusion_depth': compute_rmse(g['simple_fusion_depth'], g['gt_depth']),  # ← new
    }))
    .reset_index()
)

# Merge everything
subject_summary = subject_avg.merge(rmse_metrics, on='subject_id').reset_index(drop=True)

# Reorder columns to include the new fusion metrics
subject_summary = subject_summary[
    [
        'subject_id',
        'avg_gt_rate',
        'avg_simple_fusion_rate', 'rmse_simple_fusion_cpm',   # ← new
        'avg_fused_rate',       'rmse_fused_cpm',
        'avg_smartwatch_rate',  'rmse_smartwatch_cpm',
        'avg_video_rate',       'rmse_video_cpm',
        'avg_smartwatch_depth', 'rmse_smartwatch_depth',
        'avg_video_depth',      'rmse_video_depth',
        'avg_fused_depth',      'rmse_fused_depth',
        'avg_simple_fusion_depth', 'rmse_simple_fusion_depth',  # ← new
        'avg_gt_depth'
    ]
]

# Print and save
print(subject_summary)

subject_summary_path = f'{results_dir}/final_{test_id}_test_subject_wise_avg_rmse_rates.csv'
subject_summary.to_csv(subject_summary_path, index=False)
print(f"Saved summary to {subject_summary_path}")


Processing test id: test
   subject_id  avg_gt_rate  avg_simple_fusion_rate  rmse_simple_fusion_cpm  \
0          P0   173.908613              131.783351               55.132129   
1          P1    70.200000               84.450000               15.186343   
2         P11   130.233333              116.957143               14.136128   
3         P12   121.600000              122.466667               25.478095   
4         P14    64.125000               89.175000               25.286459   
5         P15   103.101587               97.400000               15.868523   
6         P16   117.857143              132.857143               19.209373   
7          P2   140.787879              125.272727               18.376958   
8         P20   129.200000              112.700000               19.039433   
9         P21   102.086364               97.121591               20.268885   
10        P22   120.333333              114.000000                6.749486   
11         P3    97.250000             

/tmp/ipykernel_410315/4088149155.py:78: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df


# Summary for all subjects

In [8]:
import pandas as pd
import os

# Path to your final subject summary CSV

# Get test id
fname = os.path.basename(subject_summary_path)
test_id = fname.split('_')[1]  # e.g., "3817147"
print(f"Processing test id: {test_id}")

# Load subject-level summary CSV
df = pd.read_csv(subject_summary_path)

# Clean 'subject_id' if needed
df['subject_id'] = df['subject_id'].str.strip("[]").str.replace("'", "", regex=False)

# Drop columns you don't want to average (e.g., just keep numeric columns)
numeric_cols = df.select_dtypes(include=['float', 'int']).columns

# Compute mean across all subjects
overall_avg = df[numeric_cols].mean().to_dict()

# Convert to DataFrame for saving
overall_df = pd.DataFrame([overall_avg])

# Save overall CSV
overall_output = f'{results_dir}/final_{test_id}_test_overall_avg_metrics.csv'
overall_df.to_csv(overall_output, index=False)

print("\n=== Overall average metrics ===")
for metric, value in overall_avg.items():
    print(f"{metric}: {value:.3f}")

print(f"\n✅ Saved overall averages to {overall_output}")


Processing test id: test

=== Overall average metrics ===
avg_gt_rate: 113.789
avg_simple_fusion_rate: 106.422
rmse_simple_fusion_cpm: 19.331
avg_fused_rate: 115.606
rmse_fused_cpm: 16.924
avg_smartwatch_rate: 120.152
rmse_smartwatch_cpm: 17.050
avg_video_rate: 92.692
rmse_video_cpm: 28.560
avg_smartwatch_depth: 28.654
rmse_smartwatch_depth: 12.131
avg_video_depth: 39.518
rmse_video_depth: 32.049
avg_fused_depth: 29.035
rmse_fused_depth: 11.778
avg_simple_fusion_depth: 34.086
rmse_simple_fusion_depth: 18.354
avg_gt_depth: 34.695

✅ Saved overall averages to ./results/fusion_test_999381/final_test_test_overall_avg_metrics.csv


# New Evaluation based on rules and feedback

In [9]:
# import pandas as pd
# import ast

# def rule_rate(gt_list, assistant_list):
#     result = []
#     for gt, a in zip(gt_list, assistant_list):
#         if 100 <= gt <= 120:
#             if 100 <= a <= 120:
#                 result.append('TN')
#             else:
#                 result.append('FP')
#         else:
#             if gt < 100:
#                 if a < 100:
#                     result.append('TP')
#                 else:
#                     result.append('FN')
#             else:  # gt > 120
#                 if a > 120:
#                     result.append('TP')
#                 else:
#                     result.append('FN')
#     return result

# def rule_depth(gt_list, assistant_list):
#     result = []
#     for gt, a in zip(gt_list, assistant_list):
#         if 5 <= gt <= 6:
#             if 5 <= a <= 6:
#                 result.append('TN')
#             else:
#                 result.append('FP')
#         else:
#             if gt < 5:
#                 if a < 5:
#                     result.append('TP')
#                 else:
#                     result.append('FN')
#             else:  # gt > 6
#                 if a > 6:
#                     result.append('TP')
#                 else:
#                     result.append('FN')
#     return result

# # File path
# file_path =  window_wise_results_file_path

# # Correct column names (11 columns)
# column_names = [
#     'subject_id', 'trial_id', 'start_idx', 'smartwatch_rates', 
#     'video_rates', 'gt_rates', 'smartwatch_depths', 'video_depths', 'gt_depths',
#     'smartwatch_feedback', 'video_feedback'
# ]

# # Load CSV with correct columns
# df = pd.read_csv(file_path, header=None, names=column_names)

# # Add new columns
# df['Smart Watch Hypothesis Analysis'] = ""
# df['Video Assistant Hypothesis Analysis'] = ""

# for index, row in df.iterrows():
#     try:
#         sw_rates = ast.literal_eval(row["smartwatch_rates"])
#         vi_rates = ast.literal_eval(row["video_rates"])
#         gt_rates = ast.literal_eval(row["gt_rates"])

#         sw_depths = ast.literal_eval(row["smartwatch_depths"])
#         vi_depths = ast.literal_eval(row["video_depths"])
#         gt_depths = ast.literal_eval(row["gt_depths"])

#         sw_rate_analysis = rule_rate(gt_rates, sw_rates)
#         sw_depth_analysis = rule_depth(gt_depths, sw_depths)

#         vi_rate_analysis = rule_rate(gt_rates, vi_rates)
#         vi_depth_analysis = rule_depth(gt_depths, vi_depths)

#         # Store as string (flat lists)
#         sw_analysis = sw_rate_analysis + sw_depth_analysis
#         vi_analysis = vi_rate_analysis + vi_depth_analysis

#         df.at[index, 'Smart Watch Hypothesis Analysis'] = str(sw_analysis)
#         df.at[index, 'Video Assistant Hypothesis Analysis'] = str(vi_analysis)

#     except Exception as e:
#         print(f"Error at row {index}: {e}")

# # Save final CSV only once
# output_path = file_path.replace('.csv', '_new_feedback_results.csv')
# df.to_csv(output_path, index=False)

# print(f"✅ CSV updated successfully: {output_path}")


In [10]:
# import pandas as pd
# import ast

# def rule_rate(ground_truth, comparison):
#     result = []
#     for gt, assistant in zip(ground_truth, comparison):
#         if 100 <= gt <= 120:
#             if 100 <= assistant <= 120:
#                 result.append('TN')
#             else:
#                 result.append('FP')
#         else:
#             if gt < 100:
#                 if assistant < 100:
#                     result.append('TP')
#                 else:
#                     result.append('FN')
#             else:  # gt > 120
#                 if assistant > 120:
#                     result.append('TP')
#                 else:
#                     result.append('FN')
#     return result

# def rule_depth(ground_truth, comparison):
#     result = []
#     for gt, assistant in zip(ground_truth, comparison):
#         if 5 <= gt <= 6:
#             if 5 <= assistant <= 6:
#                 result.append('TN')
#             else:
#                 result.append('FP')
#         else:
#             if gt < 5:
#                 if assistant < 5:
#                     result.append('TP')
#                 else:
#                     result.append('FN')
#             else:  # gt > 6
#                 if assistant > 6:
#                     result.append('TP')
#                 else:
#                     result.append('FN')
#     return result

# def compute_metrics(labels):
#     tp = labels.count('TP')
#     fp = labels.count('FP')
#     tn = labels.count('TN')
#     fn = labels.count('FN')

#     precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
#     recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
#     f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
#     feedback_acc = (tp + tn) / len(labels) if len(labels) > 0 else 0.0

#     return precision, recall, f1, feedback_acc


# # Correct column names (11 columns)
# column_names = [
#     'subject_id', 'trial_id', 'start_idx', 'smartwatch_rates', 
#     'video_rates', 'gt_rates', 'smartwatch_depths', 'video_depths', 'gt_depths',
#     'smartwatch_feedback', 'video_feedback'
# ]

# file_path =  window_wise_results_file_path

# # Read the CSV, explicitly set column names
# dataframe = pd.read_csv(file_path, header=None, names=column_names)

# # Add analysis columns
# dataframe['Smart Watch Hypothesis Analysis'] = None
# dataframe['Video Assistant Hypothesis Analysis'] = None

# # Store per-subject labels
# subject_labels = {}

# for index, row in dataframe.iterrows():
#     sw_compressions_rate = ast.literal_eval(row["smartwatch_rates"])
#     vi_compressions_rate = ast.literal_eval(row["video_rates"])
#     gt_rate = ast.literal_eval(row["gt_rates"])
#     sw_depth = ast.literal_eval(row["smartwatch_depths"])
#     vi_depth = ast.literal_eval(row["video_depths"])
#     gt_depth = ast.literal_eval(row["gt_depths"])

#     sw_hypothesis_rate = rule_rate(gt_rate, sw_compressions_rate)
#     sw_hypothesis_depth = rule_depth(gt_depth, sw_depth)

#     vi_hypothesis_rate = rule_rate(gt_rate, vi_compressions_rate)
#     vi_hypothesis_depth = rule_depth(gt_depth, vi_depth)

#     subject_id = row["subject_id"]

#     # Combine rate and depth hypotheses for this subject
#     if subject_id not in subject_labels:
#         subject_labels[subject_id] = []

#     subject_labels[subject_id].extend(sw_hypothesis_rate)
#     subject_labels[subject_id].extend(sw_hypothesis_depth)

#     dataframe.at[index, 'Smart Watch Hypothesis Analysis'] = str(sw_hypothesis_rate + sw_hypothesis_depth)
#     dataframe.at[index, 'Video Assistant Hypothesis Analysis'] = str(vi_hypothesis_rate + vi_hypothesis_depth)

# # Write updated batch CSV
# feedback_rule_results_path = f'{results_dir}/feedback_results_{test_id}_final.csv'
# dataframe.to_csv(feedback_rule_results_path, index=False)
# print("✅ Updated CSV with hypotheses per trial.")

# # Prepare per-subject summary metrics
# subject_metrics = []

# for subject_id, labels in subject_labels.items():
#     precision, recall, f1, feedback_acc = compute_metrics(labels)
#     subject_metrics.append({
#         'subject_id': subject_id,
#         'precision': precision,
#         'recall': recall,
#         'f1': f1,
#         'feedback_accuracy': feedback_acc
#     })

# # Create DataFrame and save to new CSV
# metrics_df = pd.DataFrame(subject_metrics)
# final_metrics_path = f'{results_dir}/final_feedback_results_{test_id}_subject_summary_metrics.csv'
# metrics_df.to_csv(final_metrics_path, index=False)
# print("✅ Created per-subject summary metrics CSV.")


In [11]:
import pandas as pd
import ast

def rule_rate(ground_truth, comparison):
    result = []
    for gt, assistant in zip(ground_truth, comparison):
        if 100 <= gt <= 120:
            if 100 <= assistant <= 120:
                result.append('TN')
            else:
                result.append('FP')
        else:
            if gt < 100:
                if assistant < 100:
                    result.append('TP')
                else:
                    result.append('FN')
            else:  # gt > 120
                if assistant > 120:
                    result.append('TP')
                else:
                    result.append('FN')
    return result

def rule_depth(ground_truth, comparison): # in millimeters [50 to 60]
    result = []
    for gt, assistant in zip(ground_truth, comparison):
        if 50 <= gt <= 60:
            if 50 <= assistant <= 60:
                result.append('TN')
            else:
                result.append('FP')
        else:
            if gt < 50:
                if assistant < 50:
                    result.append('TP')
                else:
                    result.append('FN')
            else:  # gt > 60
                if assistant > 60:
                    result.append('TP')
                else:
                    result.append('FN')
    return result

def compute_metrics(labels):
    tp = labels.count('TP')
    fp = labels.count('FP')
    tn = labels.count('TN')
    fn = labels.count('FN')

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall    = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1        = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
    feedback_acc = (tp + tn) / len(labels) if len(labels) > 0 else 0.0

    return precision, recall, f1, feedback_acc


# Correct column names (11 columns)

column_names = [
    "subject_id",
    "trial_id",
    "start_idx",
    "smartwatch_rates",
    "video_rates",
    "gt_rates",
    "fused_rates",
    "smartwatch_depths",
    "video_depths",
    "gt_depths",
    "fused_depths"
]

file_path =  window_wise_results_file_path

# Load CSV with correct columns
df = pd.read_csv(file_path, header=None, names=column_names)

# New per-row columns
df['Smart Watch Rate Analysis']     = None
df['Smart Watch Depth Analysis']    = None
df['Video Assistant Rate Analysis'] = None
df['Video Assistant Depth Analysis'] = None
# Add fused rates column
df['Fusion Rate Analysis'] = None
df['Fusion Depth Analysis'] = None

# Per-subject accumulators
subject_labels_sw_rate  = {}
subject_labels_sw_depth = {}
subject_labels_vi_rate  = {}
subject_labels_vi_depth = {}
subject_labels_fused_rate  = {}
subject_labels_fused_depth = {}
subject_labels_simple_fusion_rate = {}
subject_labels_simple_fusion_depth = {}

# add simple fusion columns 
# 
# Add new columns before loop
df['simple_fusion_rates'] = None
df['simple_fusion_depths'] = None


for idx, row in df.iterrows():
    sw_rates = ast.literal_eval(row["smartwatch_rates"])
    vi_rates = ast.literal_eval(row["video_rates"])
    gt_rates = ast.literal_eval(row["gt_rates"])
    sw_deps  = ast.literal_eval(row["smartwatch_depths"])
    vi_deps  = ast.literal_eval(row["video_depths"])
    gt_deps  = ast.literal_eval(row["gt_depths"])
    fused_rates = ast.literal_eval(row["fused_rates"])
    fused_deps  = ast.literal_eval(row["fused_depths"])


    # Compute simple fusion
    simple_fusion_rates = [0.5 * s + 0.5 * v for s, v in zip(sw_rates, vi_rates)]
    simple_fusion_depths = [0.5 * s + 0.5 * v for s, v in zip(sw_deps, vi_deps)]

    # Save as string so later code can still do ast.literal_eval
    df.at[idx, 'simple_fusion_rates'] = str(simple_fusion_rates)
    df.at[idx, 'simple_fusion_depths'] = str(simple_fusion_depths)

    print("==== DEBUG CHECK ====")
    print("Smartwatch depths:", sw_deps)
    print("Video depths:", vi_deps)
    print("Fused depths (read from file):", fused_deps)
    print("Simple fusion depths (just computed):", simple_fusion_depths)
    print("=====================")



    # remove elements where gt is greater than 220 (anomalies) for rates
    valid_indices = [i for i, gt in enumerate(gt_rates) if gt <= 220]
    gt_rates = [gt_rates[i] for i in valid_indices]
    sw_rates = [sw_rates[i] for i in valid_indices]
    vi_rates = [vi_rates[i] for i in valid_indices]
    fused_rates = [fused_rates[i] for i in valid_indices]
    simple_fusion_rates = [simple_fusion_rates[i] for i in valid_indices]

    # Get per-row labels
    sw_labels_rate  = rule_rate(gt_rates, sw_rates)
    sw_labels_depth = rule_depth(gt_deps, sw_deps)
    vi_labels_rate  = rule_rate(gt_rates, vi_rates)
    vi_labels_depth = rule_depth(gt_deps, vi_deps)
    fused_labels_rate  = rule_rate(gt_rates, fused_rates)
    fused_labels_depth = rule_depth(gt_deps, fused_deps)
    simple_fusion_labels_rate = rule_rate(gt_rates, simple_fusion_rates)
    simple_fusion_labels_depth = rule_depth(gt_deps, simple_fusion_depths)

    sid = row["subject_id"]
    # Init if not already present
    subject_labels_sw_rate.setdefault(sid, []).extend(sw_labels_rate)
    subject_labels_sw_depth.setdefault(sid, []).extend(sw_labels_depth)
    subject_labels_vi_rate.setdefault(sid, []).extend(vi_labels_rate)
    subject_labels_vi_depth.setdefault(sid, []).extend(vi_labels_depth)
    subject_labels_fused_rate.setdefault(sid, []).extend(fused_labels_rate)
    subject_labels_fused_depth.setdefault(sid, []).extend(fused_labels_depth)
    # Store simple fusion labels
    subject_labels_simple_fusion_rate.setdefault(sid, []).extend(simple_fusion_labels_rate)
    subject_labels_simple_fusion_depth.setdefault(sid, []).extend(simple_fusion_labels_depth)

    # Store per-row analyses as strings
    df.at[idx, 'Smart Watch Rate Analysis']     = str(sw_labels_rate)
    df.at[idx, 'Smart Watch Depth Analysis']    = str(sw_labels_depth)
    df.at[idx, 'Video Assistant Rate Analysis'] = str(vi_labels_rate)
    df.at[idx, 'Video Assistant Depth Analysis'] = str(vi_labels_depth)
    df.at[idx, 'Fusion Rate Analysis'] = str(fused_labels_rate)
    df.at[idx, 'Fusion Depth Analysis'] = str(fused_labels_depth)
    df.at[idx, 'simple_fusion_rate_analysis'] = str(simple_fusion_labels_rate)
    df.at[idx, 'simple_fusion_depth_analysis'] = str(simple_fusion_labels_depth)

# Save enhanced per-batch CSV
df.to_csv(f'{results_dir}/feedback_evaluation_results.csv', index=False)
print("✅ Updated CSV with separate rate/depth hypotheses per trial.")

# --- Per-subject summary metrics ---
subject_metrics = []

for sid in subject_labels_sw_rate:
    # Smartwatch rate
    sw_r_labels = subject_labels_sw_rate[sid]
    sw_r_prec, sw_r_rec, sw_r_f1, sw_r_acc = compute_metrics(sw_r_labels)
    # Smartwatch depth
    sw_d_labels = subject_labels_sw_depth[sid]
    sw_d_prec, sw_d_rec, sw_d_f1, sw_d_acc = compute_metrics(sw_d_labels)
    # Video rate
    vi_r_labels = subject_labels_vi_rate[sid]
    vi_r_prec, vi_r_rec, vi_r_f1, vi_r_acc = compute_metrics(vi_r_labels)
    # Video depth
    vi_d_labels = subject_labels_vi_depth[sid]
    vi_d_prec, vi_d_rec, vi_d_f1, vi_d_acc = compute_metrics(vi_d_labels)

    # Fused rate
    fused_r_labels = subject_labels_fused_rate[sid]
    fused_r_prec, fused_r_rec, fused_r_f1, fused_r_acc = compute_metrics(fused_r_labels)
    # Fused depth
    fused_d_labels = subject_labels_fused_depth[sid]
    fused_d_prec, fused_d_rec, fused_d_f1, fused_d_acc = compute_metrics(fused_d_labels)

    # Simple fusion rate
    simple_fusion_r_labels = subject_labels_simple_fusion_rate[sid]  # Using smartwatch labels
    simple_fusion_r_prec, simple_fusion_r_rec, simple_fusion_r_f1, simple_fusion_r_acc = compute_metrics(simple_fusion_r_labels)

    # Simple fusion depth
    simple_fusion_d_labels = subject_labels_simple_fusion_depth[sid]  # Using smartwatch labels
    simple_fusion_d_prec, simple_fusion_d_rec, simple_fusion_d_f1, simple_fusion_d_acc = compute_metrics(simple_fusion_d_labels)

    subject_metrics.append({
        'subject_id':                 sid,
        'sw_rate_precision':          sw_r_prec,
        'sw_rate_recall':             sw_r_rec,
        'sw_rate_f1':                 sw_r_f1,
        'sw_rate_feedback_accuracy':  sw_r_acc,
        'sw_depth_precision':         sw_d_prec,
        'sw_depth_recall':            sw_d_rec,
        'sw_depth_f1':                sw_d_f1,
        'sw_depth_feedback_accuracy': sw_d_acc,
        'vi_rate_precision':          vi_r_prec,
        'vi_rate_recall':             vi_r_rec,
        'vi_rate_f1':                 vi_r_f1,
        'vi_rate_feedback_accuracy':  vi_r_acc,
        'vi_depth_precision':         vi_d_prec,
        'vi_depth_recall':            vi_d_rec,
        'vi_depth_f1':                vi_d_f1,
        'vi_depth_feedback_accuracy': vi_d_acc,
        'fused_rate_precision':       fused_r_prec,
        'fused_rate_recall':          fused_r_rec,
        'fused_rate_f1':              fused_r_f1,
        'fused_rate_feedback_accuracy': fused_r_acc,
        'fused_depth_precision':      fused_d_prec,
        'fused_depth_recall':         fused_d_rec,
        'fused_depth_f1':             fused_d_f1,
        'fused_depth_feedback_accuracy': fused_d_acc,
        'simple_fusion_rate_precision': simple_fusion_r_prec,
        'simple_fusion_rate_recall': simple_fusion_r_rec,
        'simple_fusion_rate_f1': simple_fusion_r_f1,
        'simple_fusion_rate_feedback_accuracy': simple_fusion_r_acc,
        'simple_fusion_depth_precision': simple_fusion_d_prec,
        'simple_fusion_depth_recall': simple_fusion_d_rec,
        'simple_fusion_depth_f1': simple_fusion_d_f1,
        'simple_fusion_depth_feedback_accuracy': simple_fusion_d_acc
    })

metrics_df = pd.DataFrame(subject_metrics)
subject_summary_path = f'{results_dir}/feedback_subject_summary_metrics_separate_rate_depth.csv'
metrics_df.to_csv(subject_summary_path, index=False)
print("✅ Created per-subject summary metrics CSV with separate rate/depth.", subject_summary_path)


==== DEBUG CHECK ====
Smartwatch depths: [32.95095443725586, 29.445402145385742, 25.154766082763672, 25.263221740722656, 23.983346939086914, 24.378480911254883, 22.63557243347168, 25.353137969970703, 25.792781829833984, 23.26116371154785, 25.402936935424805, 23.70143699645996]
Video depths: [11.637519429922103, 175.3349479103088, 309.78777398109435, 16.36278968811035, 12.067087936401366, 80.23732964754105, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Fused depths (read from file): [32.22936245413073, 34.45794072453635, 34.92503526505766, 24.96519229930807, 23.581505495956396, 26.301469867904643, 21.86566640615508, 24.49079910967283, 24.915489318216878, 22.46997938446467, 24.53890425785017, 22.89527760935934]
Simple fusion depths (just computed): [22.29423693358898, 102.39017502784728, 167.471270031929, 20.813005714416505, 18.02521743774414, 52.307905279397964, 11.31778621673584, 12.676568984985352, 12.896390914916992, 11.630581855773926, 12.701468467712402, 11.85071849822998]
==== DEBUG CHECK ====
Sm

In [12]:
import pandas as pd

# Path to your per-subject summary metrics CSV

# Load subject-level summary
df = pd.read_csv(subject_summary_path)

print("Subject-level summary metrics:")

# Only include numeric columns for averaging
numeric_cols = df.select_dtypes(include=['float', 'int']).columns

# Compute mean of each metric
average_metrics = df[numeric_cols].mean().to_dict()

print("\n=== Overall average across all subjects ===")
for metric, value in average_metrics.items():
    print(f"{metric}: {value:.3f}")


Subject-level summary metrics:

=== Overall average across all subjects ===
sw_rate_precision: 0.708
sw_rate_recall: 0.450
sw_rate_f1: 0.529
sw_rate_feedback_accuracy: 0.666
sw_depth_precision: 0.773
sw_depth_recall: 0.981
sw_depth_f1: 0.834
sw_depth_feedback_accuracy: 0.772
vi_rate_precision: 0.550
vi_rate_recall: 0.480
vi_rate_f1: 0.470
vi_rate_feedback_accuracy: 0.516
vi_depth_precision: 0.743
vi_depth_recall: 0.769
vi_depth_f1: 0.721
vi_depth_feedback_accuracy: 0.630
fused_rate_precision: 0.630
fused_rate_recall: 0.494
fused_rate_f1: 0.525
fused_rate_feedback_accuracy: 0.627
fused_depth_precision: 0.773
fused_depth_recall: 0.981
fused_depth_f1: 0.834
fused_depth_feedback_accuracy: 0.772
simple_fusion_rate_precision: 0.616
simple_fusion_rate_recall: 0.534
simple_fusion_rate_f1: 0.544
simple_fusion_rate_feedback_accuracy: 0.617
simple_fusion_depth_precision: 0.753
simple_fusion_depth_recall: 0.839
simple_fusion_depth_f1: 0.762
simple_fusion_depth_feedback_accuracy: 0.680


In [13]:
import pandas as pd
import ast
import numpy as np

# Path to your CSV file
csv_file = "./results/debug_752743_smartwatch_model_test_fusion_debug_per_batch.csv"  # <-- replace with your actual file

# Define columns explicitly since there's no header
columns = [
    "subject_id",
    "trial_id",
    "start_idx",
    "smartwatch_rates",
    "video_rates",
    "gt_rates",
    "fused_rates",
    "smartwatch_depths",
    "video_depths",
    "gt_depths",
    "fused_depths"
]

# Read CSV with no header and assign columns
df = pd.read_csv(csv_file, header=None, names=columns)

# Function to compute RMSE and MAE
def compute_metrics(gt, pred):
    gt = np.array(gt)
    pred = np.array(pred)
    rmse = np.sqrt(np.mean((gt - pred) ** 2))
    mae = np.mean(np.abs(gt - pred))
    return rmse, mae

# List to store per-trial metrics
trial_results = []

for idx, row in df.iterrows():
    subject = row['subject_id']
    trial = row['trial_id']

    # Parse string arrays
    gt_rates = ast.literal_eval(row['gt_rates'])
    sw_rates = ast.literal_eval(row['smartwatch_rates'])
    vid_rates = ast.literal_eval(row['video_rates'])
    fused_rates = ast.literal_eval(row['fused_rates'])

    gt_depths = ast.literal_eval(row['gt_depths'])
    sw_depths = ast.literal_eval(row['smartwatch_depths'])
    vid_depths = ast.literal_eval(row['video_depths'])
    fused_depths = ast.literal_eval(row['fused_depths'])

    # remove elements where gt is greater than 195 (anomalies) for rates
    valid_indices = [i for i, gt in enumerate(gt_rates) if gt <= 195]
    gt_rates = [gt_rates[i] for i in valid_indices]
    sw_rates = [sw_rates[i] for i in valid_indices]
    vid_rates = [vid_rates[i] for i in valid_indices]
    fused_rates = [fused_rates[i] for i in valid_indices]


    # Compute metrics
    rmse_sw_rate, mae_sw_rate = compute_metrics(gt_rates, sw_rates)
    rmse_vid_rate, mae_vid_rate = compute_metrics(gt_rates, vid_rates)
    rmse_fused_rate, mae_fused_rate = compute_metrics(gt_rates, fused_rates)

    rmse_sw_depth, mae_sw_depth = compute_metrics(gt_depths, sw_depths)
    rmse_vid_depth, mae_vid_depth = compute_metrics(gt_depths, vid_depths)
    rmse_fused_depth, mae_fused_depth = compute_metrics(gt_depths, fused_depths)

    # Store per-trial
    trial_results.append({
        "subject_id": subject,
        "rmse_sw_rate": rmse_sw_rate,
        "mae_sw_rate": mae_sw_rate,
        "rmse_vid_rate": rmse_vid_rate,
        "mae_vid_rate": mae_vid_rate,
        "rmse_fused_rate": rmse_fused_rate,
        "mae_fused_rate": mae_fused_rate,
        "rmse_sw_depth": rmse_sw_depth,
        "mae_sw_depth": mae_sw_depth,
        "rmse_vid_depth": rmse_vid_depth,
        "mae_vid_depth": mae_vid_depth,
        "rmse_fused_depth": rmse_fused_depth,
        "mae_fused_depth": mae_fused_depth,
    })

# Convert to DataFrame
trial_df = pd.DataFrame(trial_results)

# Group by subject and average
subject_df = trial_df.groupby("subject_id").mean().reset_index()


# Save results
subject_df.to_csv("./results/752743_subject_trial_metrics.csv", index=False)

print("Per-subject metrics saved to '752743_subject_trial_metrics.csv'.")

FileNotFoundError: [Errno 2] No such file or directory: './results/debug_752743_smartwatch_model_test_fusion_debug_per_batch.csv'